# 💡 这节课会带给你

1. 系统性维护、测试、监控一个 LLM 应用
2. 学习使用主流的工具完成上述工作

## 维护一个生产级的 LLM 应用，我们需要做什么？

1. 各种指标监控与统计：访问记录、响应时长、Token用量、计费等等
2. 调试 Prompt
3. 测试/验证系统的相关评估指标
4. 数据集管理（便于回归测试）
5. Prompt 版本管理（便于升级/回滚）

## 针对以上需求，我们介绍三个生产级 LLM App 维护平台

1. 重点讲解 **LangFuse**: 开源 + SaaS，LangSmith 平替，可集成 LangChain 也可直接对接 OpenAI API；
2. 简单讲解 **LangSmith**: LangChain 的官方平台，SaaS 服务，非开源
3. 简单讲解 **Prompt Flow**：微软开发，开源 + Azure AI云服务，可集成 Semantic Kernel（但貌合神离）。

## 1、LangFuse

开源，支持 LangChain 集成或原生 OpenAI API 集成

官方网站：https://langfuse.com/

项目地址：https://github.com/langfuse

1. 通过官方云服务使用：注册: cloud.langfuse.com，创建 API Key，并添加如下配置
```sh
LANGFUSE_SECRET_KEY="sk-lf-..."
LANGFUSE_PUBLIC_KEY="pk-lf-..."
# 官方服务器
LANGFUSE_HOST="https://us.cloud.langfuse.com" 
```

2. 通过 Docker 本地部署

```sh
# Clone repository
git clone https://github.com/langfuse/langfuse.git
cd langfuse
 
# Run server and db
docker compose up -d
```

In [ ]:
!pip install --upgrade langfuse

### 1.1 通过 LangChain 的回调集成

In [1]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from langfuse.callback import CallbackHandler

# 定义回调处理器
langfuse_handler = CallbackHandler()

# 检查是否可用，返回True即可用
langfuse_handler.auth_check()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

from langchain.schema import HumanMessage
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("Say hello to {input}!") 
])


# 定义输出解析器
parser = StrOutputParser()

# 定义Chain
chain = (
    {"input":RunnablePassthrough()} 
    | prompt
    | model
    | parser
)

chain.invoke(input="AGIClass", config={"callbacks":[langfuse_handler]})

'Hello, AGIClass! Excited to engage with you! How can I assist you today?'

### 1.2、Trace
- 一个Trace相当于一轮对话
- 用 Trace 可以记录一个多次调用 LLM 的过程


 以下面应用为例：

**AGI课堂跟课助手**，根据课程内容，判断学生问题是否需要老师解答

1. 判断该问题是否需要老师解答，回复'Y'或'N'
2. 判断该问题是否已有同学问过

In [11]:
# 构建 PromptTemplate
from langchain.prompts import PromptTemplate

need_answer_prompt=PromptTemplate.from_template("""
*********
你是AIGC课程的助教，你的工作是从学员的课堂交流中选择出需要老师回答的问题，加以整理以交给老师回答。
 
课程内容:
{outlines}
*********
学员输入:
{user_input}
*********
如果这是一个需要老师答疑的问题，回复Y，否则回复N。
只回复Y或N，不要回复其他内容。""")

check_duplicated_prompt=PromptTemplate.from_template("""
*********
已有提问列表:
[
{question_list}
]
*********
新提问:
{user_input}
*********
已有提问列表是否有和新提问类似的问题? 回复Y或N, Y表示有，N表示没有。
只回复Y或N，不要回复其他内容。""")

In [12]:
outlines="""
LangChain
模型 I/O 封装
模型的封装
模型的输入输出
PromptTemplate
OutputParser
数据连接封装
文档加载器：Document Loaders
文档处理器
内置RAG：RetrievalQA
记忆封装：Memory
链架构：Chain/LCEL
大模型时代的软件架构：Agent
ReAct
SelfAskWithSearch
Assistants API
LangServe
LangChain.js
"""

question_list=[
    "谢谢老师",
    "LangChain开源吗",
]

In [14]:
# 创建 chain
model = ChatOpenAI(temperature=0)
parser = StrOutputParser()

chain1 = (
    need_answer_prompt
    | model
    | parser
)

chain2 = (
    check_duplicated_prompt
    | model
    | parser
)

In [15]:
import uuid
from langfuse.client import Langfuse

# 创建一个新trace
def create_trace(user_id):
    langfuse = Langfuse()
    trace_id = str(uuid.uuid4())
    trace = langfuse.trace(name="assistant", id=trace_id, user_id=user_id)
    return trace

# 主流程
def verify_question(question: str, outlines: str, question_list: list, user_id: str) -> bool:
    trace = create_trace(user_id)
    handler = trace.get_langchain_handler()
    # 判断是否需要回答
    if chain1.invoke({"user_input":question,"outlines": outlines},config={"callbacks":[handler]}) == 'Y':
        # 判断是否为重复问题
        if chain2.invoke({"user_input":question,"question_list": "\n".join(question_list)},config={"callbacks":[handler]}) == 'N':
            question_list.append(question)
            return True
    return False

In [16]:
# 实际调用
ret = verify_question(
    "LangChain支持Java吗",
    outlines,
    question_list,
    user_id="fxy",
)
print(ret)

True


<img src="langfuse1.png" width="1000px">

<img src="trace-detail.png" width="1000px">

### 1.3、Session

- 用 Session 记录一个用户的多轮对话

In [17]:
import uuid
from langchain_openai import ChatOpenAI
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

llm = ChatOpenAI()

messages = [
    SystemMessage(content="你是AGIClass的课程助理。"), 
]

handler = CallbackHandler(user_id="fxy", trace_name="session_test",session_id=str(uuid.uuid4()))

while True:
    user_input=input("User: ")
    if user_input.strip() == "":
        break
    messages.append(HumanMessage(content=user_input))
    response = llm.invoke(messages,config={"callbacks":[handler]})
    print("AI: "+response.content)
    messages.append(response)

User: 你好
AI: 你好！有什么我可以帮助你的吗？
User: 你可以干什么？
AI: 我可以帮助你解答问题、提供学习资源、解释课程内容、协助作业或项目，以及提供一般性的学习建议。如果你有具体的需求或问题，请告诉我！
User: 


<img src="session.png" width="1000px">

### 1.4、数据集与测试

- 利用用户聊天数据集和回归测试功能不断优化我们的大模型代码
- [Langfuse Datasets Cookbook](https://langfuse.com/docs/datasets/python-cookbook)

### 1.4.1、在线标注

1. 创建一个数据集

<img src="create_dataset.png" width="1000px">

2. 将一条Trace加入数据集

<img src="add_to_dataset.png" width="1000px">

### 1.4.2、上传已有数据集

In [9]:
import json

qa_pairs = []
with open('example_dataset.jsonl','r',encoding='utf-8') as fp:
    for line in fp:
        example = json.loads(line.strip())
        qa_pairs.append(example)


from langfuse import Langfuse
from langfuse.model import CreateDatasetRequest, CreateDatasetItemRequest
 
# init
langfuse = Langfuse()

langfuse.create_dataset(name="wiki_qa")

for item in qa_pairs[:20]:
  langfuse.create_dataset_item(
        dataset_name="wiki_qa",
        input=item["question"],
        expected_output=item["answer"]
)


<img src="lf_upload_data.png" width=1000px>

### 1.4.3、定义评估函数


如何比较两个句子的相似性：一些经典 NLP 的评测方法（选）

1. **编辑距离**：也叫莱文斯坦距离(Levenshtein),是针对二个字符串的差异程度的量化量测，量测方式是看至少需要多少次的处理才能将一个字符串变成另一个字符串。
   - 具体计算过程是一个动态规划算法：https://zhuanlan.zhihu.com/p/164599274
   - 衡量两个句子的相似度时，可以以词为单位计算
2. **BLEU Score**:
   - 它通过比较机器生成的文本与一个或多个参考文本，衡量生成文本与参考文本在**词汇**层面上的相似性。BLEU仅基于词汇重合度进行评估，不考虑文本的语义，因此即使生成文本在意义上是正确的，但如果在词汇选择上不同，分数可能仍然较低。
   - 计算输出与参照句之间的 n-gram 准确率（n=1...4）
   - 对短输出做惩罚
   - 在整个测试集上平均下述值
   - 完整计算公式：$\mathrm{BLEU}_4=\min\left(1,\frac{output-length}{reference-length}\right)\left(\prod_{i=1}^4 precision_i\right)^{\frac{1}{4}}$
   - 函数库：https://www.nltk.org/_modules/nltk/translate/bleu_score.html
3. **Rouge Score**:
   - Rouge-N：将模型生成的结果和标准结果按 N-gram 拆分后，只计算召回率；
   - Rouge-L: 利用了最长公共子序列（Longest Common Sequence），计算：$P=\frac{LCS(c,r)}{len(c)}$, $R=\frac{LCS(c,r)}{len(r)}$, $F=\frac{(1+\beta^2)PR}{R+\beta^2P}$
   - 函数库：https://pypi.org/project/rouge-score/
   - 对比 BLEU 与 ROUGE：
     - BLEU 能评估流畅度，但指标偏向于较短的翻译结果（brevity penalty 没有想象中那么强）
     - ROUGE 不管流畅度，所以只适合深度学习的生成模型：结果都是流畅的前提下，ROUGE 反应参照句中多少内容被生成的句子包含（召回）
5. **METEOR**: 另一个从机器翻译领域借鉴的指标。与 BLEU 相比，METEOR 考虑了更多的因素，如同义词匹配、词干匹配、词序等，因此它通常被认为是一个更全面的评价指标。
   - 对语言学和语义词表有依赖，所以对语言依赖强。

下面代码使用NLTK(Natural Language Toolkit)工具计算bleu_score

In [12]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import re

def bleu_score(output, expected_output):
    def _tokenize(sentence):
        # 正则表达式定义了要去除的标点符号
        return re.sub(r'[^\w\s]', '', sentence.lower()).split()
    score = sentence_bleu(
        [_tokenize(expected_output)], 
        _tokenize(output), 
        smoothing_function=SmoothingFunction().method3
    )
    print(score)
    return score

### 1.4.4、运行测试

Prompt 模板与 Chain（LCEL）

In [4]:
from langchain.retrievers import WikipediaRetriever
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """
Answer user's question according to the context below. 
Be brief, answer in no more than 20 words.
CONTEXT_START
{context}
CONTEXT_END

USER QUESTION:
{input}
"""


# 定义语言模型
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

# 定义Prompt模板
prompt = PromptTemplate.from_template(prompt_template)

# 检索 wikipedia
retriever = WikipediaRetriever(top_k_results=1)


# 定义输出解析器
parser = StrOutputParser()

wiki_qa_chain = (
    {
        "context": retriever, 
        "input": RunnablePassthrough()
    } 
    | prompt
    | llm
    | parser
)

在数据集上测试效果

In [14]:
dataset = langfuse.get_dataset("wiki_qa")
 
for item in dataset.items:
    # Langchain callback handler that automatically links the execution trace to the dataset item
    handler = item.get_langchain_handler(run_name="test_wiki_qa-20")
 
    # Execute application and pass custom handler
    output = wiki_qa_chain.invoke(item.input, config={"callbacks":[handler]})
    
    handler.trace.score(
      name="bleu_score",
      value=bleu_score(output, item.expected_output)
    )
 
# Flush the langfuse client to ensure all data is sent to the server at the end of the experiment run
langfuse.flush()

0.010439917202667988
0.019852949681089915
0.008854366238409026
0.027627972078128442
0.03938557432578257
0.06299033575335412
0.10454786294785588
0.034120260193708875
0.030266181377145762
0.01049627690924115
0.046214670445444084
0.027142536025659754
0.023154722339652548
0.09379548093903634
0.13285541256864652
0.015758838106836467
0.029970944654446528
0.09446965843281002
0.05795599612995367
0.02286956778061901


<img src="lf_dataset_run.png" width=1000px>

### 1.5、Prompt 版本管理

[使用文档](https://langfuse.com/docs/prompts/get-started)

<img src="prompt_management.png" width="1000px">

In [15]:
from langfuse import Langfuse
from langchain_core.prompts import PromptTemplate
 

langfuse = Langfuse()
 
print("============下面是最新production版本==============")
langfuse_prompt = langfuse.get_prompt("need_answer")
 
langchain_prompt = PromptTemplate.from_template(langfuse_prompt.get_langchain_prompt())
print(langchain_prompt.format(outlines="test", user_input="这是我的输入"))


print("============下面获取指定版本版本==============")
langfuse_prompt = langfuse.get_prompt("need_answer", version=1)
 
langchain_prompt = PromptTemplate.from_template(langfuse_prompt.get_langchain_prompt())
print(langchain_prompt.format(outlines="test", user_input="这是我的输入"))


============下面是最新production版本==============
*********
你是AIGC课程的助教，你的工作是从学员的课堂交流中选择出需要老师回答的问题，加以整理以交给老师回答。

你的选择需要遵循以下原则：
1 需要老师回答的问题是指与课程内容或AI/LLM相关的技术问题；
2 评论性的观点、闲聊、表达模糊不清的句子，不需要老师回答；
3 学生输入不构成疑问句的，不需要老师回答；
4 学生问题中如果用“这”、“那”等代词指代，不算表达模糊不清，请根据问题内容判断是否需要老师回答。
 
课程内容:
test
*********
学员输入:
这是我的输入
*********
Analyse the student's input according to the lecture's contents and your criteria.
Output your analysis process step by step.
Finally, output a single letter Y or N in a separate line.
Y means that the input needs to be answered by the teacher.
N means that the input does not needs to be answered by the teacher.
============下面获取指定版本版本==============
*********
你是AIGC课程的助教，你的工作是从学员的课堂交流中选择出需要老师回答的问题，加以整理以交给老师回答。
 
课程内容:
test
*********
学员输入:
这是我的输入
*********
如果这是一个需要老师答疑的问题，回复Y，否则回复N。
只回复Y或N，不要回复其他内容。""")


## 2、LangSmith

平台入口：https://www.langchain.com/langsmith

文档地址：https://python.langchain.com/docs/langsmith/walkthrough

将你的 LangChain 应用与 LangSmith 链接，需要：

1. 注册账号，并申请一个`LANGCHAIN_API_KEY`
2. 在环境变量中设置以下值

```
LANGCHAIN_TRACING_V2=true
LANGCHAIN_PROJECT=YOUR_PROJECT_NAME #自定义项目名称
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com #LangSmith的服务端点
LANGCHAIN_API_KEY=LANGCHAIN_API_KEY # LangChain API Key
```
3. 程序中的调用将自动被记录

In [2]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 定义语言模型
llm = ChatOpenAI()

# 定义Prompt模板
prompt = PromptTemplate.from_template("Say hello to {input}!")

# 定义输出解析器
parser = StrOutputParser()

chain = (
    {"input":RunnablePassthrough()} 
    | prompt
    | llm
    | parser
)

chain.invoke("AGIClass")

'Hello AGIClass! How can I assist you today?'

<img src="langsmith-example.png" width="1000px">

### 2.2、数据集管理与测试

### 2.2.1、在线标注演示
1、创建数据集

<img src="ls_create_dataset.png" width="1000px">

2、添加到数据集
<img src="ls_add_to_dataset.png" width="1000px">


### 2.2.2、上传数据集

In [16]:
from langsmith import Client
import json


data = []
with open('example_dataset.jsonl','r',encoding='utf-8') as fp:
    for line in fp:
        example = json.loads(line.strip())
        data.append(example)
        
        client = Client()

# 创建数据集
dataset_name = "wiki-qa"
dataset = client.create_dataset(
    dataset_name, #数据集名称
    description="wiki问答数据集", #数据集描述
)

client.create_examples(
    inputs=[{"input":item["question"]} for item in data[:20]], 
    outputs=[{"output":item["answer"]} for item in data[:20]], 
    dataset_id=dataset.id
)

<img src="ls_upload_data.png" width="1000px">

### 2.2.3、评估函数

LanChain内置3种评估器：

（1）String Evaluators: 通过将生成的输出（预测）与参考字符串或输入进行比较来评估语言模型的性能。https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/string/

EnvaluatorType: https://api.python.langchain.com/en/latest/evaluation/langchain.evaluation.schema.EvaluatorType.html

（2）Trajectory Evaluators：用于评估agent的行为表现。https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/trajectory/


（3）Comparison Evaluators：用于评估不同模型的输出结果，常用于两个语言模型之间的A/B测试，或者比较同一模型的不同版本。https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/
    
    

下面的代码是一个自定义的StringEvaluator

In [19]:
from langchain.evaluation import StringEvaluator
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import re
from typing import Optional, Any

class BleuEvaluator(StringEvaluator):

    def __init__(self):
        pass

    @property
    def requires_input(self) -> bool:
        return False

    @property
    def requires_reference(self) -> bool:
        return True

    @property
    def evaluation_name(self) -> str:
        return "bleu_score"

    def _tokenize(self,sentence):
        # 正则表达式定义了要去除的标点符号
        return re.sub(r'[^\w\s]', '', sentence.lower()).split()
    
    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        bleu_score = sentence_bleu(
            [self._tokenize(reference)], 
            self._tokenize(prediction), 
            smoothing_function=SmoothingFunction().method3
        )
        return {"score": bleu_score}

In [24]:
from langchain.evaluation import EvaluatorType
from langchain.smith import RunEvalConfig

evaluation_config = RunEvalConfig(
    # 评估器，可多选
    evaluators=[
        # 使用一个内置的StringEvaluator
        EvaluatorType.QA,
    ],
    # 可追加自定评估标准
    custom_evaluators=[BleuEvaluator()]
)

### 2.2.4、运行测试

In [ ]:
!pip install wikipedia

In [25]:
from langchain.prompts import PromptTemplate
from langchain.retrievers import WikipediaRetriever
from operator import itemgetter


prompt = PromptTemplate.from_template("""
*********
Answer user's question according to the context below. 
Be brief, answer in no more than 20 words.
CONTEXT_START
{context}
CONTEXT_END

USER QUESTION:
{input}

""")

# 检索 wikipedia
retriever = WikipediaRetriever(top_k_results=1)

model = ChatOpenAI(temperature=0)

parser = StrOutputParser()

chain_v1 = (
    {
        "context": itemgetter("input") | retriever | (lambda docs: "\n".join([doc.page_content for doc in docs])),
        "input":  itemgetter("input")
     }
    | prompt
    | model
    | parser
)

In [26]:
from langchain.smith import arun_on_dataset, run_on_dataset

from uuid import uuid4

unique_id = uuid4().hex[0:8]

results = await arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain_v1,
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"LangChain_WikiQA_Project-{unique_id}"
)

View the evaluation results for project 'LangChain_WikiQA_Project-d79243cd' at:
https://smith.langchain.com/o/101a6386-9727-5197-958a-5868b750060a/datasets/404233ce-938e-4648-b8d2-3d21c394c3b6/compare?selectedSessions=8a664c71-cf73-4be2-b3dc-7d1018334f0e

View all tests for Dataset wiki-qa at:
https://smith.langchain.com/o/101a6386-9727-5197-958a-5868b750060a/datasets/404233ce-938e-4648-b8d2-3d21c394c3b6
[------------------------------------------------->] 20/20

,feedback.bleu_score,error,execution_time,run_id
count,20.000000,0,20.000000,20
unique,NaN,0,NaN,20
top,NaN,NaN,NaN,19fcca60-b329-4157-b258-4a3041b5e4cf
freq,NaN,NaN,NaN,1
mean,0.049011,NaN,10.188564,NaN
std,0.054358,NaN,2.064550,NaN
min,0.010058,NaN,7.599712,NaN
25%,0.019579,NaN,8.757824,NaN
50%,0.030168,NaN,9.573589,NaN
75%,0.062321,NaN,11.923407,NaN


<img src="ls_dataset_run.png" width="1000px">